# Übung 6: Transfer learning

Neuronal Netze benötigen oft eine große Menge an Trainingsdaten, damit es nicht zu overfitting kommt. Transfer learning erlaubt es, mit relativ geringen Datenmenge dennoch erfolgreiche große Netze zu trainieren. Dabei verwendet man ein bereits auf einen anderen Datensatz (z.b. ImageNet) vortrainiertes Netzwerk, und ersetzt nur das letzte Layer durch ein neues. In dieser Übung geht es darum, ein Netzwerk für die Erkennung von Geparden und Leoparden in der freien Wildbahn zu trainineren. 

## Daten laden

Lade die Daten hier herunter: http://tonic.imp.fu-berlin.de/cv_data/data.tar.gz

Die Daten wurde bereits in ein Trainings- und Validierungsset geteilt. Die Ordnerstruktur ist wie bei vielen Bildklassifierungsdatensetzen so aufgebaut. Es gibt zwei Unterordner für die Trainings- und Validierunsdaten. In diesen Ordnern liegen dann jeweils alle Bilder von einer Klasse in einem Unterordner mit dem Namen der Klasse.

Ein Beispiel: Die Trainingsbilder für die Klasse "cheetah" liegen in dem Unterordner train/cheetah

Diese Orderstruktur wird auch von dem in keras enhaltenen ImageDataGenerator unterstützt.

In [101]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"]="-1"
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session, get_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

from keras.preprocessing.image import ImageDataGenerator
#
from keras.models import Sequential
from keras.layers import Dense, Flatten, Convolution2D, MaxPooling2D, Dropout
from keras.optimizers import RMSprop
from keras.datasets import mnist
from keras.utils import np_utils
from keras import initializers
from keras import backend as K
from keras.applications import resnet50
from keras.models import load_model
from keras.models import Model

from PIL import Image

In [103]:
batch_size = 32
image_input_size = (224, 224)
data_path = './data/'

#geaddet
prob_drop_conv = 0.5                # drop probability for dropout @ conv layer
pool_size = (2, 2)                  # size of pooling area for max pooling
nb_epoch = 100
opt = RMSprop(lr=0.000001, rho=0.9)

In [107]:
train_data_path = os.path.join(data_path, 'train')
val_data_path = os.path.join(data_path, 'val')

izw_classes = ('unknown', 'cheetah', 'leopard')

generator = ImageDataGenerator(horizontal_flip=True)
val_generator = ImageDataGenerator(horizontal_flip=False)

train_gen = generator.flow_from_directory(
    train_data_path, 
    target_size=image_input_size,
    classes=izw_classes,
    batch_size=batch_size)

val_gen = val_generator.flow_from_directory(
    val_data_path, 
    target_size=image_input_size,
    classes=izw_classes,
    batch_size=batch_size)


Found 17857 images belonging to 3 classes.
Found 1916 images belonging to 3 classes.


## Training ohne transfer learning

Trainiere zuerst ein kleines Classifer-Netzwerk ohne transfer learning. Falls du keine Grafikkarte hast, solltest du nicht die volle Auflösung (siehe Variable image_input_size) verwenden, da das Training sonst zu lange dauert. Eine Bildgröße von 32x32 Pixeln wäre zum Beispiel möglich.

In [106]:

#das convolutional net tutorial von keras

# Convolutional model
model = Sequential()

# conv1 layer
model.add(Convolution2D(8, 3, 3, border_mode='same', activation='relu', input_shape=[224,224,3]))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), border_mode='same'))
model.add(Dropout(prob_drop_conv))

# conv2 layer
model.add(Convolution2D(16, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), border_mode='same'))
model.add(Dropout(prob_drop_conv))

# conv3 layer
model.add(Convolution2D(4, 3, 3, border_mode='same', activation='relu'))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2), border_mode='same'))
model.add(Flatten())
model.add(Dropout(prob_drop_conv))

# fc1 layer
model.add(Dense(625, activation='relu'))
model.add(Dropout(prob_drop_conv))

# fc2 layer
model.add(Dense(3, activation='softmax'))

model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train
history = model.fit_generator(train_gen, nb_epoch=nb_epoch, shuffle=True, verbose=1)



c:\users\t\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(8, (3, 3), input_shape=[224, 224,..., padding="same", activation="relu")`
  
c:\users\t\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(strides=(2, 2), padding="same", pool_size=(2, 2))`
  if __name__ == '__main__':
c:\users\t\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(16, (3, 3), padding="same", activation="relu")`
  del sys.path[0]
c:\users\t\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:14: UserWarning: Update your `MaxPooling2D` call to the Keras 2 API: `MaxPooling2D(strides=(2, 2), padding="same", pool_size=(2, 2))`
  
c:\users\t\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:18: UserWarning: Update your `Co

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 224, 224, 8)       224       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 112, 112, 8)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 112, 112, 8)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 16)      1168      
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 56, 56, 16)        0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 56, 56, 16)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 56, 56, 4)         580       
__________

KeyboardInterrupt: 

In [40]:
# Evaluate
#evaluation = model.evaluate_generator(val_gen, verbose=1)
evaluation = model.evaluate_generator(val_gen, 1)

print('Summary: Loss over the test dataset: %.2f, Accuracy: %.2f' % (evaluation[0], evaluation[1]))

Summary: Loss over the test dataset: 3.53, Accuracy: 0.78


Erstelle eine Confusion matrix basierend auf den Ausgaben des Netzes für die Validierungsdaten und berechne den ROC AUC für die Klasse cheetah. Du kannst hierfür optional die scikit-learn Bibliothek verwenden.

In [ ]:
# TODO

## Pretrained network

Lade nun ein auf Imagenet vortrainiertes Netzwerk und klassifiziere damit die Validierungsdaten. Eine Anleitung für keras findest du hier: https://keras.io/applications

Du kannst selber entscheiden, welche Netzwerkarchitektur du verwendest.

In [94]:
model2 = resnet50.ResNet50(weights = 'imagenet')
model2.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_7[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [79]:
evaluation2 = model2.predict_generator(val_gen, verbose=1)

60/60 [==============================] - 18s 296ms/step


In [80]:
#TODO DELETE THIS
model2.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [81]:
pred = resnet50.decode_predictions(evaluation2, top=1)
print('Predicted:', pred)


Predicted: [[('n03388043', 'fountain', 0.97630036)], [('n03388043', 'fountain', 0.8123213)], [('n02115641', 'dingo', 0.34068754)], [('n03388043', 'fountain', 0.44861355)], [('n03388043', 'fountain', 0.98608786)], [('n02130308', 'cheetah', 0.9469464)], [('n03388043', 'fountain', 0.9887434)], [('n03388043', 'fountain', 0.96982026)], [('n03388043', 'fountain', 0.24596648)], [('n04589890', 'window_screen', 0.14704761)], [('n03388043', 'fountain', 0.59126544)], [('n03388043', 'fountain', 0.7378707)], [('n03388043', 'fountain', 0.9815662)], [('n03891251', 'park_bench', 0.1988769)], [('n02130308', 'cheetah', 0.96657157)], [('n04589890', 'window_screen', 0.3041777)], [('n04371774', 'swing', 0.15579952)], [('n04589890', 'window_screen', 0.1606817)], [('n03388043', 'fountain', 0.99919766)], [('n02130308', 'cheetah', 0.43335074)], [('n02130308', 'cheetah', 0.849557)], [('n03388043', 'fountain', 0.6657455)], [('n01855672', 'goose', 0.17976525)], [('n07248320', 'book_jacket', 0.15170856)], [('n0389

In [82]:
#TODO DELETE THIS
model2.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

Da der ImageNet-Datensatz auch die Klassen cheetah und leopard enthält, können wir sogar ohne transfer learning das vortrainierte Netzwerk evaluieren. Interpretiere alle Klassen außer cheetah und leopard als unknown und berechne wie im vorherigen Schritt die Confusion matrix und den ROC AUC score für die Klasse cheetah.

In [83]:
for img in pred:
    if img[0][1] not in izw_classes:
        img[0] = (img[0][0],'unknown',img[0][2])

for img in pred:
    print(img)

[('n03388043', 'unknown', 0.97630036)]
[('n03388043', 'unknown', 0.8123213)]
[('n02115641', 'unknown', 0.34068754)]
[('n03388043', 'unknown', 0.44861355)]
[('n03388043', 'unknown', 0.98608786)]
[('n02130308', 'cheetah', 0.9469464)]
[('n03388043', 'unknown', 0.9887434)]
[('n03388043', 'unknown', 0.96982026)]
[('n03388043', 'unknown', 0.24596648)]
[('n04589890', 'unknown', 0.14704761)]
[('n03388043', 'unknown', 0.59126544)]
[('n03388043', 'unknown', 0.7378707)]
[('n03388043', 'unknown', 0.9815662)]
[('n03891251', 'unknown', 0.1988769)]
[('n02130308', 'cheetah', 0.96657157)]
[('n04589890', 'unknown', 0.3041777)]
[('n04371774', 'unknown', 0.15579952)]
[('n04589890', 'unknown', 0.1606817)]
[('n03388043', 'unknown', 0.99919766)]
[('n02130308', 'cheetah', 0.43335074)]
[('n02130308', 'cheetah', 0.849557)]
[('n03388043', 'unknown', 0.6657455)]
[('n01855672', 'unknown', 0.17976525)]
[('n07248320', 'unknown', 0.15170856)]
[('n03891251', 'unknown', 0.23785082)]
[('n03891251', 'unknown', 0.41405636

[('n03788365', 'unknown', 0.30611527)]
[('n02410509', 'unknown', 0.5171183)]
[('n02130308', 'cheetah', 0.91323704)]
[('n02410509', 'unknown', 0.2335557)]
[('n03388043', 'unknown', 0.69763774)]
[('n03388043', 'unknown', 0.6418716)]
[('n03388043', 'unknown', 0.93414426)]
[('n02130308', 'cheetah', 0.6533024)]
[('n02504458', 'unknown', 0.5302287)]
[('n03388043', 'unknown', 0.92798495)]
[('n03388043', 'unknown', 0.9940572)]
[('n02655020', 'unknown', 0.4141666)]
[('n02130308', 'cheetah', 0.9241108)]
[('n03891251', 'unknown', 0.15305218)]
[('n02231487', 'unknown', 0.49475393)]
[('n01748264', 'unknown', 0.3790424)]
[('n02130308', 'cheetah', 0.9721147)]
[('n03788365', 'unknown', 0.74010456)]
[('n03788365', 'unknown', 0.2599158)]
[('n02130308', 'cheetah', 0.91587687)]
[('n03388043', 'unknown', 0.9983748)]
[('n02130308', 'cheetah', 0.9593595)]
[('n02130308', 'cheetah', 0.91392374)]
[('n03388043', 'unknown', 0.8197281)]
[('n03388043', 'unknown', 0.9465535)]
[('n03388043', 'unknown', 0.52502996)]
[

## Transfer learning

Das vortrainierte Netzwerk kann nun mit unseren Daten weitertrainiert werden. Ersetze dafür das letzte Layer in dem Netzwerk mit einem Dense Layer mit 3 Ausgaben für unsere Klassen cheetah, leopard und unknown. Du kannst selbst entscheiden, ob du nun das komplette Netzwerk mit trainierst oder nur das neu eingefügte, letzte Layer.

Auch hierfür kannst du dich wieder an der keras Anleitung orientieren: https://keras.io/applications

In [84]:
#TODO DELETE THIS
model2.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [95]:
for layer in model2.layers:
    layer.trainable = False
model2.layers.pop()
model2.layers.pop()
model2.summary()

model2.layers[-1].outbound_nodes = []
model2.outputs = [model2.layers[-1].output]
output = model2.get_layer('avg_pool').output
output = Flatten()(output)
output = Dense(output_dim=3)(output)
model3 = Model(model2.input,output)
#model2.save_weights('my_model_weights.h5')
#model2.save('test.h5')

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_7[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

c:\users\t\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:11: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=3)`
  # This is added back by InteractiveShellApp.init_path()


In [96]:
model3.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_7[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

Evaluiere das so trainierte Netzwerk wie in den letzten beiden Aufgaben.

In [109]:
#WARUM FUNKTIONIERT DAS SO NICHT? :-(
model3.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
history2 = model3.fit_generator(train_gen, steps_per_epoch=100, nb_epoch=nb_epoch, shuffle=True, verbose=1)

c:\users\t\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
c:\users\t\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras.pre..., verbose=1, shuffle=True, epochs=100, steps_per_epoch=100)`
  


Epoch 1/100


FailedPreconditionError: Attempting to use uninitialized value bn4a_branch2b_6/moving_mean
	 [[Node: bn4a_branch2b_6/moving_mean/read = Identity[T=DT_FLOAT, _class=["loc:@bn4a_branch2b_6/moving_mean"], _device="/job:localhost/replica:0/task:0/device:GPU:0"](bn4a_branch2b_6/moving_mean)]]
	 [[Node: loss_3/mul/_319 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_3257_loss_3/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'bn4a_branch2b_6/moving_mean/read', defined at:
  File "c:\users\t\anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\t\anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\t\anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, 

# Auswertung

Beschreibe kurz qualitativ die Resultate. Wie unterscheiden sich die trainierten Netzwerke, zum Beispiel im Bezug auf die Genauigkeit oder die Laufzeit? Welche Entscheidungen musstest du bei der Erfüllung der Aufgaben treffen und warum hast du dich für den von dir gewählten Weg entschieden?